### Getting data - Headlines

In [ ]:
import pandas as pd

full_data = pd.read_json('input/Sarcasm_Headlines_Dataset.json', lines=True)

full_data = full_data.drop(['article_link'], axis=1)
full_data = full_data.rename(index=str, columns={'headline': 'content', 'is_sarcastic': 'sarcastic'})

print(full_data.head())

### Getting data - Reddit

In [ ]:
df_reddit = pd.read_csv('input/train-balanced-sarcasm.csv')

df_reddit['content'] = df_reddit['comment']
#df_reddit['content'] = df_reddit['parent_comment'].astype(str).str.cat(df_reddit['comment'].astype(str), sep='. ')
df_reddit = df_reddit.drop(['author', 'ups', 'score', 'downs', 'date', 'created_utc', 'subreddit', 'parent_comment', 'comment'], axis=1)
df_reddit = df_reddit.rename(index=str, columns={'label': 'sarcastic'})

print(df_reddit.head())

### Concatenating data

In [ ]:
print(len(df_reddit))
print(len(full_data))

full_data_final = pd.concat([full_data, df_reddit[:]])

print(len(full_data_final))
print(full_data_final.head())

### Droping NaN values and reseting index

In [ ]:
full_data_final.dropna(subset=['content'], inplace=True)
full_data_final = full_data_final.reset_index(drop=True)
print(full_data_final['sarcastic'].value_counts())

### Balancing data

In [ ]:
labels_neg = full_data_final['sarcastic'].value_counts()[0]
labels_sar = full_data_final['sarcastic'].value_counts()[1]

imbalance_count = abs(labels_neg - labels_sar)
print(imbalance_count)

to_add = []

added = 0
if labels_sar < labels_neg:
    for index, row in df_reddit[100001:].iterrows():
        if row['sarcastic'] == 1:
            to_add.append((row['content'], row['sarcastic']))
            added += 1
            if added >= imbalance_count:
                break
else:
    for index, row in df_reddit[100001:].iterrows():
        if row['sarcastic'] == 0:
            to_add.append((row['content'], row['sarcastic']))
            added += 1
            if added >= imbalance_count:
                break

df_add = pd.DataFrame(to_add)
df_add = df_add.rename(index=str, columns={0: 'content', 1: 'sarcastic'})
print((df_add.head()))

full_data_final = pd.concat([full_data_final, df_add])

print('Balanced:')
print(full_data_final['sarcastic'].value_counts())

### Saving data to .csv file

In [ ]:
full_data_final.to_csv('input/full_data_final.csv')